In [9]:
import os

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [10]:
from src.backend.orchestrator import Orchestrator

orch = Orchestrator()
debug_message = orch.initialise_conversation()
print(debug_message)

INFO:custom_logger:[__init__] Orchestrator instance created.
INFO:custom_logger:[initialise_conversation] Initializing conversation with system instruction.
INFO:custom_logger:[initialise_conversation] Conversation initialized: {'role': 'system', 'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for

{'role': 'system', 'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as state

In [11]:
debug_message["content"]

'\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stated by user.\n    All the values

In [12]:
debug_message

{'role': 'system',
 'content': '\n    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.\n    You need to ask relevant questions and understand the user profile by analysing the user\'s responses.\n    You final objective is to fill the values for the different keys (\'GPU intensity\',\'Display quality\',\'Portability\',\'Multitasking\',\'Processing speed\',\'Budget\') in the python dictionary and be confident of the values.\n    These key value pairs define the user\'s profile.\n    The python dictionary looks like this\n    {\'GPU intensity\': \'values\',\'Display quality\': \'values\',\'Portability\': \'values\',\'Multitasking\': \'values\',\'Processing speed\': \'values\',\'Budget\': \'values\'}\n    The value for \'Budget\' should be a numerical value extracted from the user\'s response.\n    The values for all keys, except \'Budget\', should be \'low\', \'medium\', or \'high\' based on the importance of the corresponding keys, as stat

In [1]:
import pandas as pd
df = pd.read_parquet('src/database/laptop_data_mapped.parquet')

In [13]:
df['mapped_dictionary'][1]

{'Display quality': 'medium',
 'GPU intensity': 'low',
 'Multitasking': 'low',
 'Portability': 'medium',
 'Processing speed': 'medium'}

In [3]:
from llama_index.experimental.query_engine import PandasQueryEngine

In [4]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
result = query_engine.query(
    """
    Give me list of 3 laptop that match this requirements:
    
    {'Display quality': 'medium',
    'GPU intensity': 'low',
    'Multitasking': 'low',
    'Portability': 'medium',
    'Processing speed': 'medium',
    'Budget': 50,000}
    """
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df[(df['mapped_dictionary'].apply(lambda x: x.get('Display quality') == 'medium')) & 
   (df['mapped_dictionary'].apply(lambda x: x.get('GPU intensity') == 'low')) & 
   (df['mapped_dictionary'].apply(lambda x: x.get('Multitasking') == 'low')) & 
   (df['mapped_dictionary'].apply(lambda x: x.get('Portability') == 'medium')) & 
   (df['mapped_dictionary'].apply(lambda x: x.get('Processing speed') == 'medium')) & 
   (df['Price'].str.replace(',', '').astype(int) <= 50000)].head(3)
```
> Pandas Output:     Brand Model Name Core CPU Manufacturer Clock Speed RAM Size Storage Type  \
0    Dell   Inspiron   i5            Intel     2.4 GHz      8GB          SSD   
3  Lenovo    IdeaPad   i3            Intel     2.1 GHz      8GB          HDD   
8      HP   Pavilion   i5            Intel     2.3 GHz   

In [10]:
type(result)

llama_index.core.base.response.schema.Response

In [18]:
result.response

'  Brand Model Name Core CPU Manufacturer Clock Speed RAM Size Storage Type  \\\n0  Dell   Inspiron   i5            Intel     2.4 GHz      8GB          SSD   \n1   MSI       GL65   i7            Intel     2.6 GHz     16GB      HDD+SSD   \n2    HP  EliteBook   i7            Intel     2.8 GHz     16GB          SSD   \n\n  Display Type Display Size Graphics Processor Screen Resolution          OS  \\\n0          LCD        15.6"          Intel UHD         1920x1080  Windows 10   \n1          IPS        15.6"         NVIDIA GTX         1920x1080  Windows 10   \n2          LED          14"          Intel UHD         1920x1080  Windows 11   \n\n  Laptop Weight    Special Features Warranty Average Battery Life   Price  \\\n0        2.5 kg    Backlit Keyboard   1 year              6 hours  35,000   \n1        2.3 kg        RGB Keyboard  2 years              4 hours  55,000   \n2        1.5 kg  Fingerprint Sensor  3 years              8 hours  90,000   \n\n                                      